## Recomendação de Filmes com Similaridade de Conteúdo - NLTK

Este notebook demonstra um sistema de recomendação de filmes com base em similaridade de conteúdo. O código realiza as seguintes etapas:

1. Pré-processamento de Dados: Carregamento e fusão de conjuntos de dados de filmes e créditos.
2. Engenharia de Recursos: Extração de informações relevantes e normalização de dados.
3. Análise de Linguagem Natural: Transformação de descrições de filmes em vetores de recursos.
4. Recomendação de Filmes: Geração de recomendações com base na similaridade.
5. Exibição de Recomendações: Demonstração das recomendações para filmes específicos.


In [57]:
import pandas as pd
import numpy as np
import ast

In [58]:
# Carregando os DataFrames
df_filmes = pd.read_csv('data/movies.csv', low_memory=False)
df_creditos = pd.read_csv('data/credits.csv')

### Pré processamento de dados

In [59]:
# Exibição dos DataFrames
print(df_filmes.info())
print(df_creditos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [60]:
# Concateção dos DataFrames
df_filmes = df_filmes.merge(df_creditos, on='title') # on='title' -> chave de junção
df_filmes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4808 entries, 0 to 4807
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4808 non-null   int64  
 1   genres                4808 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4808 non-null   int64  
 4   keywords              4808 non-null   object 
 5   original_language     4808 non-null   object 
 6   original_title        4808 non-null   object 
 7   overview              4805 non-null   object 
 8   popularity            4808 non-null   float64
 9   production_companies  4808 non-null   object 
 10  production_countries  4808 non-null   object 
 11  release_date          4807 non-null   object 
 12  revenue               4808 non-null   int64  
 13  runtime               4806 non-null   float64
 14  spoken_languages      4808 non-null   object 
 15  status               

In [61]:
# Seleção de colunas relevantes
df_filmes = df_filmes[['movie_id','title', 'overview','genres', 'cast', 'keywords', 'crew']] 

In [62]:
# Verificação de valores nulos e remoção
print(df_filmes.isnull().sum())
df_filmes.dropna(inplace=True)

# Presemça de valores duplicados
print('Valores duplicados: ', df_filmes.duplicated().sum())


movie_id    0
title       0
overview    3
genres      0
cast        0
keywords    0
crew        0
dtype: int64
Valores duplicados:  0


In [63]:
# Tratamento para converter as colunas em listas
def convert_col_to_list(col):
    """Converte a coluna em uma lista"""
    if isinstance(col, list):
        return [i['name'] for i in col]
    else:
        return []

df_filmes['genres'] = df_filmes['genres'].apply(ast.literal_eval).apply(convert_col_to_list) # ast.literal_eval -> converte a string em lista
df_filmes['cast'] = df_filmes['cast'].apply(ast.literal_eval).apply(convert_col_to_list)
df_filmes['keywords'] = df_filmes['keywords'].apply(ast.literal_eval).apply(convert_col_to_list)

# Filtra o nome do diretor
def get_diretor(col):
    """Filtra o nome do diretor"""
    for i in col:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

df_filmes['director'] = df_filmes['crew'].apply(ast.literal_eval).apply(get_diretor)
df_filmes.drop('crew', axis=1, inplace=True)

In [64]:
# Título em maiusculo
df_filmes['title'] = df_filmes['title'].str.upper()

In [65]:
# Conversão de overview e director para lista
df_filmes['overview'] = df_filmes['overview'].apply(lambda x: x.split())
df_filmes['director'] = df_filmes['director'].apply(lambda x: [x])

In [66]:
# Remoção de espaços em branco
def remove_space(col):
    """Remove os espaços em branco"""
    if isinstance(col, list):
        return [str.lower(i.replace(" ", "")) for i in col]
    else:
        if isinstance(col, str):
            return str.lower(col.replace(" ", ""))
        else:
            return ''

# Aplicação da função
df_filmes['genres'] = df_filmes['genres'].apply(remove_space)
df_filmes['cast'] = df_filmes['cast'].apply(remove_space)
df_filmes['keywords'] = df_filmes['keywords'].apply(remove_space)
df_filmes['director'] = df_filmes['director'].apply(remove_space) if isinstance(df_filmes['director'], str) else df_filmes['director']


In [67]:
# Criação de uma coluna com os dados concatenados
df_filmes['metadata'] = df_filmes['overview'] + df_filmes['genres'] + df_filmes['keywords'] + df_filmes['cast'] + df_filmes['director']

In [68]:
# Cria um novo DataFrame com as colunas relevantes
df_filmes = df_filmes[['movie_id','title','metadata']]

In [69]:
# Restauração dos espaços em branco
df_filmes['metadata'] = df_filmes['metadata'].apply(lambda x: ' '.join(str(i) for i in x)) # str(i) -> converte o valor em string para poder usar o join

In [70]:
# Transformação dos dados para minúsculo
df_filmes['metadata'] = df_filmes['metadata'].str.lower()

### Análise de linguagem natural

In [71]:
from sklearn.feature_extraction.text import CountVectorizer # CountVectorizer -> cria uma matriz de contagem de token
from sklearn.metrics.pairwise import cosine_similarity # cosine_similarity -> calcula a similaridade entre os vetores

count_vec = CountVectorizer(max_features= 5000 ,stop_words='english') # stop_words='english' -> remove as palavras 'inúteis' em inglês
vetores = count_vec.fit_transform(df_filmes['metadata']).toarray() # fit_transform -> transforma os dados em uma matriz de contagem de token

In [72]:
# Uso de linguagem natural para encontrar a similaridade entre os filmes
from nltk.stem.porter import PorterStemmer # PorterStemmer -> reduz as palavras para o seu radical

port_strem = PorterStemmer()

def stemmer(col):
    """Reduz as palavras para o seu radical"""
    return [port_strem.stem(i) for i in col]

# Aplicação da função
df_filmes['metadata'] = df_filmes['metadata'].apply(lambda x: stemmer(x.split()))

In [83]:
similaridade = cosine_similarity(vetores) # Calcula a similaridade entre os vetores

def normalize_similarity_score(score):
    # Normaliza a pontuação para uma escala de 0 a 100
    normalized_score = (score + 1) * 50
    return normalized_score


# Ordenação dos filmes por similaridade
def recomendacao_filmes_classificadas(filme):
    filme = filme.upper()
    if filme not in df_filmes['title'].unique():
        return [('Filme não encontrado', 0)]
    else:
        index_filme = df_filmes.loc[df_filmes['title'] == filme].index[0]
        filmes_similares = sorted(enumerate(similaridade[index_filme]), key=lambda x: x[1], reverse=True)[1:11]
        recommendations = []
        for i, sim in filmes_similares:
            title = df_filmes.iloc[i]['title']
            score = normalize_similarity_score(sim)
            recommendations.append((title, score))
        return recommendations

# Exibir os filmes recomendados
def exibir_recomendacoes(filme):
    recomendacoes = recomendacao_filmes_classificadas(filme)
    if recomendacoes[0][0] == 'Filme não encontrado':
        print(recomendacoes[0][0])
    else:
        for movie, score in recomendacoes:
            print(f"{movie}, Pontuação de Similariedade: {score:.2f}")



### Sugestões de filmes

In [84]:
# Exibição dos filmes recomendados para o filme Shrek
exibir_recomendacoes('Shrek')

SHREK 2, Pontuação de Similariedade: 67.43
ALADDIN, Pontuação de Similariedade: 66.18
SHREK FOREVER AFTER, Pontuação de Similariedade: 62.99
FROZEN, Pontuação de Similariedade: 61.92
SHANGHAI NOON, Pontuação de Similariedade: 61.80
MULAN, Pontuação de Similariedade: 61.69
SHREK THE THIRD, Pontuação de Similariedade: 61.57
ASTERIX AT THE OLYMPIC GAMES, Pontuação de Similariedade: 61.32
PUSS IN BOOTS, Pontuação de Similariedade: 61.25
THE PRINCESS BRIDE, Pontuação de Similariedade: 61.21


In [85]:
# Exibição dos filmes recomendados para o filme The Dark Knight
exibir_recomendacoes('The Dark Knight')

BATMAN BEGINS, Pontuação de Similariedade: 70.56
THE DARK KNIGHT RISES, Pontuação de Similariedade: 70.16
BATMAN RETURNS, Pontuação de Similariedade: 63.34
BATMAN FOREVER, Pontuação de Similariedade: 61.69
BATMAN, Pontuação de Similariedade: 61.40
AMIDST THE DEVIL'S WINGS, Pontuação de Similariedade: 61.40
BATMAN & ROBIN, Pontuação de Similariedade: 61.30
GANGSTER'S PARADISE: JERUSALEMA, Pontuação de Similariedade: 60.47
BATMAN: THE DARK KNIGHT RETURNS, PART 2, Pontuação de Similariedade: 60.40
NINE QUEENS, Pontuação de Similariedade: 59.97


In [86]:
# Exibição dos filmes recomendados para o filme The Godfather
exibir_recomendacoes('The Godfather')

DESERT DANCER, Pontuação de Similariedade: 76.37
TAKE THE LEAD, Pontuação de Similariedade: 70.40
STEP UP, Pontuação de Similariedade: 67.84
CENTER STAGE, Pontuação de Similariedade: 67.40
STEP UP 2: THE STREETS, Pontuação de Similariedade: 67.40
ABCD (ANY BODY CAN DANCE), Pontuação de Similariedade: 67.37
TANGO, Pontuação de Similariedade: 66.71
STEP UP REVOLUTION, Pontuação de Similariedade: 65.84
FOOTLOOSE, Pontuação de Similariedade: 65.52
DANCIN' IT'S ON, Pontuação de Similariedade: 64.02
